In [1]:
import numpy as np
import pandas as pd
import os
import torch
from sklearn.model_selection import train_test_split
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.datasets import MemoryMapDataset

# EDA полученных датасетов, 4000 общих client_id в каждом

In [2]:
dir = 'D:/Файлы/Хакатон_июнь/'  # Путь к директории с файлами

In [3]:
dial = pd.read_parquet(dir + "selected4000_client_ids_dial_test.parquet")
dial.shape

(14155, 3)

In [4]:
dial.head(2)

,client_id,event_time,embedding
0,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-27 12:58:16.172754,"[0.32905003, -0.30735606, 0.4486652, -0.423453..."
1,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-17 13:39:52.447299,"[0.16543284, 0.15017991, 0.11980294, 0.3943023..."


In [5]:
arr = dial.client_id.unique()
arr

array(['00c3b97963114e1c65d2e226db266bfa2242ac0f5a81034ea891bcdbf23f10d2',
       '91515e8cb104510a9c3d5b618cf31953a1a4689700abd5477728ed77c6c392f5',
       '00f110a9587a2fe928ca7e4dd0bb7991419d9e74fc4617a51edabd8ce6cbc22c',
       ...,
       '4b6792b40df909fca40a4881f08f62f8e189b98029b6ce932cf4603432f0eadd',
       '55b009e762cea8c258b332994c1a4a17834616f63f4285b098acc4d3c07268be',
       'd912f568f7dc878abf4a9996133bf13798053da8e0da6653e93addd4988cb26a'],
      dtype=object)

## Исследование геометок клиентов

In [6]:
geo = pd.read_parquet(dir + "selected4000_client_ids_geo_test.parquet")
geo.shape

(2012523, 5)

In [7]:
geo.head(2)

,client_id,event_time,geohash_4,geohash_5,geohash_6
0,a84a895385789e2ff3b3331c6ecd46ce1ebc91c64777c7...,2022-04-02 06:06:22.497484,17071,49241,1309213
1,a84a895385789e2ff3b3331c6ecd46ce1ebc91c64777c7...,2022-09-01 04:12:46.946580,17071,100702,1417072


In [8]:
geo.nunique()  # Каша из геометок

client_id        2411
event_time    2012523
geohash_4        5130
geohash_5       21152
geohash_6       79594
dtype: int64

In [9]:
geo_filtered1 = geo[geo['client_id'] == '00c3b97963114e1c65d2e226db266bfa2242ac0f5a81034ea891bcdbf23f10d2']  # посмотрим одного клиента
geo_filtered1

,client_id,event_time,geohash_4,geohash_5,geohash_6
10369,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-09-03 05:22:26.313750,34832,73322,1880396
10370,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-10 09:46:01.446959,34832,73322,2104618
10371,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-04-21 04:42:42.630529,34832,73322,1880396
10372,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-01 10:42:11.525827,34832,73322,2104618
10373,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-31 04:46:59.539763,34832,73322,1880396
...,...,...,...,...,...
11575,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-10-25 10:10:29.589796,34832,73322,1270529
11576,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-04-17 05:26:22.828483,34832,73322,1270529
11577,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-04-18 05:17:02.149251,34832,73322,1270529
11578,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-04-29 12:50:24.100832,34832,73322,1270529


In [10]:
geo_filtered1.nunique()  # количество уникальных геометок клиента. Наверное, все эти геометки касаются банкоматов и магазинов

client_id        1
event_time    1211
geohash_4        3
geohash_5        7
geohash_6       30
dtype: int64

In [11]:
geo_filtered2 = geo[geo['client_id'] == '91515e8cb104510a9c3d5b618cf31953a1a4689700abd5477728ed77c6c392f5']  # посмотрим второго клиента
geo_filtered2  # нет данных для этого клиента

,client_id,event_time,geohash_4,geohash_5,geohash_6


In [12]:
geo.client_id.nunique()  # прикольно, не все геометки есть для клиентов. Видимо, фильтрация прошла не очень удачно

2411

Можно вообще опустить геометки, и не включать их в модель. Обоснование - не понятно, чем они являются.
Это могут быть как географические координаты, так и обычные метки банкоматов и кассовых аппаратов.
Можно, конечно, попробовать сравнить геометки с транзакциями.
Но тогда будет ясно, что, например, один вид товара дорогой, а другой - не очень.
И дорогие товары покупаются в местах с одними геометкам, а более дешёвые - в местах с другми геометками. 

Думаю, опустим геометки. Сосредоточимся на транзакциях и диалогах клиентов, и попробуем привязать таргет к этим сущностям

## Исследование транзакций клиентов

In [13]:
trx = pd.read_parquet(dir + "selected4000_client_ids_trx_test.parquet")
trx.shape

(830539, 14)

In [14]:
trx.client_id.nunique()  # тоже не все данные попали, хахаха!

3072

In [15]:
trx_filtered1 = trx[trx['client_id'] == '00c3b97963114e1c65d2e226db266bfa2242ac0f5a81034ea891bcdbf23f10d2']  # посмотрим одного клиента
trx_filtered1  # Оказалось меньше транзакций клиента, чем геометок

,event_time,amount,client_id,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
1267,2022-09-01 13:34:18.607865,7.511817e+04,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,869.0,17340.0,26661.0,87.0,1780.0,47.0
1268,2022-06-18 14:50:48.894812,4.812308e+05,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,56,9,11.0,19.0,344.0,869.0,17340.0,26661.0,87.0,1780.0,47.0
1269,2022-07-05 05:14:22.610815,3.117521e+04,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,364.0,22652.0,26661.0,87.0,1780.0,47.0
1270,2022-10-28 14:02:21.617460,1.227057e+06,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,56,9,11.0,19.0,344.0,869.0,17340.0,26661.0,87.0,1780.0,47.0
1271,2022-08-27 09:20:44.597712,3.154246e+05,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,364.0,22652.0,26661.0,87.0,1780.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,2022-10-27 15:16:58.291550,3.926654e+04,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,869.0,17340.0,26661.0,87.0,1780.0,47.0
1430,2022-09-14 12:50:00.194770,2.286886e+05,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,869.0,31488.0,26661.0,87.0,1780.0,47.0
1431,2022-09-24 20:06:34.510673,1.271642e+04,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,869.0,31488.0,26661.0,87.0,1780.0,47.0
1432,2022-04-30 11:35:01.004117,4.594338e+02,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,54,55,11.0,19.0,344.0,364.0,22652.0,26661.0,87.0,1780.0,47.0


In [16]:
trx_filtered1.nunique()  # только для одного клиента

event_time       167
amount           167
client_id          1
event_type         5
event_subtype      5
currency           1
src_type11         1
src_type12         1
dst_type11         2
dst_type12         3
src_type21         1
src_type22         1
src_type31         1
src_type32         1
dtype: int64

In [17]:
trx_filtered1.describe()

,event_time,amount,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
count,167,1.670000e+02,167.000000,167.00000,167.0,167.0,167.0,167.000000,167.000000,167.0,167.0,167.0,167.0
mean,2022-06-23 19:14:20.861437,1.884973e+05,52.850299,50.42515,11.0,19.0,344.0,732.922156,21736.526946,26661.0,87.0,1780.0,47.0
min,2022-03-06 14:14:00.806397,2.860226e+02,20.000000,9.00000,11.0,19.0,344.0,364.000000,17340.000000,26661.0,87.0,1780.0,47.0
25%,2022-04-29 02:30:40.082101,1.954535e+04,54.000000,55.00000,11.0,19.0,344.0,364.000000,17340.000000,26661.0,87.0,1780.0,47.0
50%,2022-06-10 15:06:14.486335,6.952621e+04,54.000000,55.00000,11.0,19.0,344.0,869.000000,17340.000000,26661.0,87.0,1780.0,47.0
75%,2022-08-26 02:59:39.101533,2.164540e+05,54.000000,55.00000,11.0,19.0,344.0,869.000000,22652.000000,26661.0,87.0,1780.0,47.0
max,2022-10-29 14:28:40.733912,1.890680e+06,56.000000,55.00000,11.0,19.0,344.0,869.000000,31488.000000,26661.0,87.0,1780.0,47.0
std,NaN,3.091518e+05,4.626572,10.99858,0.0,0.0,0.0,224.732183,5514.258137,0.0,0.0,0.0,0.0


Транзакции имеют знак +, и никакой другой. Значит, есть данные только потому, как клиент платит

In [18]:
trx.nunique()  # для всех клиентских транзакций. Целых пять типов валют, жесть
# Типы и субтипы событий выглядят избыточно. Можно оставить в модели только event_subtype

event_time       830539
amount           827573
client_id          3072
event_type           52
event_subtype        55
currency              5
src_type11           24
src_type12           90
dst_type11           32
dst_type12          124
src_type21         1100
src_type22           82
src_type31          630
src_type32           78
dtype: int64

## Диалоги клиентов

In [19]:
dial = pd.read_parquet(dir + "selected4000_client_ids_dial_test.parquet")
dial.head(2)

,client_id,event_time,embedding
0,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-27 12:58:16.172754,"[0.32905003, -0.30735606, 0.4486652, -0.423453..."
1,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-17 13:39:52.447299,"[0.16543284, 0.15017991, 0.11980294, 0.3943023..."


In [20]:
dial.client_id.nunique()

4000

In [21]:
dial.event_time.nunique()  # у каждого диалога - уникальное время

14155

In [22]:
dial.shape

(14155, 3)

In [23]:
dial.loc[:, 'embedding_length'] = dial['embedding'].apply(len)  # попробуем оценить длину эмбеддингов
dial.embedding_length.nunique()  # Что, всего один элемент в списке длин?

1

In [24]:
dial  # реально, все эмбеддинги имеют одну длину

,client_id,event_time,embedding,embedding_length
0,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-27 12:58:16.172754,"[0.32905003, -0.30735606, 0.4486652, -0.423453...",768
1,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-17 13:39:52.447299,"[0.16543284, 0.15017991, 0.11980294, 0.3943023...",768
2,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-12 12:54:19.688310,"[0.089870565, 0.010766345, 0.3462482, 0.115493...",768
3,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-07-16 11:20:39.410252,"[0.12595204, -0.13321288, 0.3942836, -0.008274...",768
4,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-29 10:21:25.952550,"[0.13154733, -0.0046007102, 0.24747911, -0.115...",768
...,...,...,...,...
14150,35dc68423fc3479729c218beff269eb37c7ef2483fc411...,2022-01-10 16:18:08.934007,"[0.06768528, -0.069106236, 0.18480805, 0.06712...",768
14151,969083192404e7da10257507721c10fb5eee2e45d7a2cf...,2022-10-01 14:23:22.626918,"[0.21046256, -0.030083066, 0.49536, -0.3427803...",768
14152,d47c9167733faac306b9445b9e5f0a143f3f03ed2e6109...,2022-12-27 04:12:11.645326,"[-0.18810052, 0.04221305, 0.1811354, 0.3640972...",768
14153,d47c9167733faac306b9445b9e5f0a143f3f03ed2e6109...,2022-12-22 03:17:41.826068,"[0.49085075, -0.36978078, 0.64939636, -0.79797...",768


In [25]:
dial.loc[:, 'embedding_sum'] = dial['embedding'].apply(np.sum)  # оценим сумму элементов эмбеддингов
dial  # суммы разные. Значит, все эти эмбеддинги можно как-то преобразовать в простую величину, и попробовать оценить влияние величины на покупки

,client_id,event_time,embedding,embedding_length,embedding_sum
0,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-27 12:58:16.172754,"[0.32905003, -0.30735606, 0.4486652, -0.423453...",768,12.945056
1,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-17 13:39:52.447299,"[0.16543284, 0.15017991, 0.11980294, 0.3943023...",768,8.760626
2,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-12 12:54:19.688310,"[0.089870565, 0.010766345, 0.3462482, 0.115493...",768,6.772972
3,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-07-16 11:20:39.410252,"[0.12595204, -0.13321288, 0.3942836, -0.008274...",768,10.594102
4,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-29 10:21:25.952550,"[0.13154733, -0.0046007102, 0.24747911, -0.115...",768,13.071451
...,...,...,...,...,...
14150,35dc68423fc3479729c218beff269eb37c7ef2483fc411...,2022-01-10 16:18:08.934007,"[0.06768528, -0.069106236, 0.18480805, 0.06712...",768,10.233727
14151,969083192404e7da10257507721c10fb5eee2e45d7a2cf...,2022-10-01 14:23:22.626918,"[0.21046256, -0.030083066, 0.49536, -0.3427803...",768,14.892079
14152,d47c9167733faac306b9445b9e5f0a143f3f03ed2e6109...,2022-12-27 04:12:11.645326,"[-0.18810052, 0.04221305, 0.1811354, 0.3640972...",768,7.193572
14153,d47c9167733faac306b9445b9e5f0a143f3f03ed2e6109...,2022-12-22 03:17:41.826068,"[0.49085075, -0.36978078, 0.64939636, -0.79797...",768,17.598486


In [26]:
# Надеюсь, что это диалоги клиента непосредственно с банком, иначе зачем это всё?
dial = dial.drop(['embedding', 'embedding_length'], axis=1)  # Отбросим эти векторные данные
dial

,client_id,event_time,embedding_sum
0,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-27 12:58:16.172754,12.945056
1,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-17 13:39:52.447299,8.760626
2,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-12 12:54:19.688310,6.772972
3,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-07-16 11:20:39.410252,10.594102
4,91515e8cb104510a9c3d5b618cf31953a1a4689700abd5...,2022-06-29 10:21:25.952550,13.071451
...,...,...,...
14150,35dc68423fc3479729c218beff269eb37c7ef2483fc411...,2022-01-10 16:18:08.934007,10.233727
14151,969083192404e7da10257507721c10fb5eee2e45d7a2cf...,2022-10-01 14:23:22.626918,14.892079
14152,d47c9167733faac306b9445b9e5f0a143f3f03ed2e6109...,2022-12-27 04:12:11.645326,7.193572
14153,d47c9167733faac306b9445b9e5f0a143f3f03ed2e6109...,2022-12-22 03:17:41.826068,17.598486


In [27]:
# dial.to_parquet("D:/Файлы/Хакатон_июнь/selected4000_client_ids_dial_test_processed.parquet", index=False)

## Изучаем таргет

In [28]:
target = pd.read_parquet(dir + "selected4000_client_ids_test_target_b.parquet")
target.head(2)

,mon,target_1,target_2,target_3,target_4,client_id
0,2022-11-30,0,0,0,0,d89a0712b64d779d9c383298c0cfe0dc27bdf824b9d253...
1,2022-05-31,0,0,0,0,b1567a620a7525794cd6b7c97259c95a5f8203412040c6...


In [29]:
target.shape

(21797, 6)

In [30]:
target.nunique()

mon            11
target_1        2
target_2        2
target_3        2
target_4        2
client_id    2174
dtype: int64

Данные таргета также подобраны не для всех. Только для 2174 уникальных клиентов.

In [31]:
a = 21797 / 2174  # Однако для каждого клиента есть данные за 10 месяцев
a

10.026218951241951

In [32]:
print(target.target_1.unique(), target.target_2.unique(), target.target_3.unique(), target.target_4.unique())  # таргеты - категориальные переменные
# не предсказать ли эти таргеты как тензор?

[0 1] [0 1] [0 1] [0 1]


In [33]:
target.mon.unique()  # А это - описание дат. Видимо, к этим датам состоялась покупка таргета.

array(['2022-11-30', '2022-05-31', '2022-07-31', '2022-03-31',
       '2022-04-30', '2022-02-28', '2022-08-31', '2022-10-31',
       '2022-09-30', '2022-06-30', '2022-12-31'], dtype=object)

In [34]:
# Купил ли один клиент несколько продуков банка? Купил ли он несколько раз один продукт?
# Судя по всему, некоторые продкуты покупаются не так уж и часто
print(target.target_1.sum(), target.target_2.sum(), target.target_3.sum(), target.target_4.sum())

743 53 536 282


In [35]:
target_1_counts = target.groupby('client_id')['target_1'].sum().value_counts()
target_1_counts  # некоторые клиенты покупают три раза, некоторые - два раза, но чаще только один раз. И ещё чаще не покупают вовсе

target_1
0    1497
1     614
2      60
3       3
Name: count, dtype: int64

In [36]:
target_2_counts = target.groupby('client_id')['target_2'].sum().value_counts()
target_2_counts

target_2
0    2126
1      44
2       3
3       1
Name: count, dtype: int64

In [37]:
target_3_counts = target.groupby('client_id')['target_3'].sum().value_counts()
target_3_counts

target_3
0    1674
1     467
2      30
3       3
Name: count, dtype: int64

In [38]:
target_4_counts = target.groupby('client_id')['target_4'].sum().value_counts()
target_4_counts  # кто-то готов купить и 4, и 5 раз. Но это "вылеты".

target_4
0    1921
1     231
2      18
3       2
5       1
4       1
Name: count, dtype: int64

# Преобразование данных и подготовка к моделированию

Возможно, клиент совершает покупку банковского продукта почти сразу, как принял решение сделать её. В наше время люди редко имеют возможность обдумывать такие задачи неделями. Следовательно, какие-то паттерны поведения предшествуют принятию решения о покупке продукта и самой покупке. Этими паттернами могут быть учащения покупок вообще (больше trx в месяц, когда совершалась покупка таргета), больше трат (больше amount в месяц, связанный с таргетом), какие-то особые покупки, имеющие как банковский признак (например, самый частый src_type11 и другие самые частые признаки транзакций) количества дополнительных признаков транзакций (например, сумма всех src_type11 и других признаков транзакций минус 1), и медиана сумм эмбеддингов (embedding_sum) диалогов клиента, а также количество диалогов клиента за месяц до совершения покупки банковского продукта.

Интересно, что продуты банка можно купить и не один раз. Что это вообще за продукты такие?

Таргет лучше преобразовать в тензор.

Таким образом, предстоит работа по сбору всех признаков в одну таблицу, причём каждая строка будет иметь данные о паттерне поведения клиента в месяц, который предшествует дате покупки продукта.

In [39]:
target['mon'] = pd.to_datetime(target['mon'])  # выделим данные о месяце, когда оценивался статус таргета
target['month'] = target['mon'].dt.month.astype(str).str.zfill(2)
target_mon = target.copy()
target_mon.head(1)

,mon,target_1,target_2,target_3,target_4,client_id,month
0,2022-11-30,0,0,0,0,d89a0712b64d779d9c383298c0cfe0dc27bdf824b9d253...,11


In [40]:
dial['event_time'] = pd.to_datetime(dial['event_time'])  # выделим данные о месяце, когда случился разговор 
dial['event_month'] = dial['event_time'].dt.month.astype(str).str.zfill(2)
dial_mon = dial.copy()
dial_mon.head(1)

,client_id,event_time,embedding_sum,event_month
0,00c3b97963114e1c65d2e226db266bfa2242ac0f5a8103...,2022-03-27 12:58:16.172754,12.945056,03


In [41]:
dial_mon = dial_mon.drop(['event_time'], axis=1)

In [42]:
dial_mon.shape  # много разговоров. Никуда не годится

(14155, 3)

In [43]:
dial_mon_median = dial_mon.groupby(['client_id', 'event_month'])['embedding_sum'].median().reset_index()
dial_mon_median.shape  # Конечно, есть разговоры клиентов не только за те месяцы, когда оценивался таргет.

(8542, 3)

In [44]:
dial_mon_median.head(1)

,client_id,event_month,embedding_sum
0,00124cd2582ddd2b0916648aa5f4c5af6c14f26efa6068...,09,12.523173


In [45]:
dial_mon_median.groupby('event_month')['client_id'].nunique()
# Есть фактор лета - начала осени? Интересно, что именно в жто время ведут больше разговоров

event_month
01     444
02     358
03     469
04     468
05     580
06     955
07    1145
08    1091
09    1053
10     996
11     668
12     315
Name: client_id, dtype: int64

In [46]:
target_dial = target.merge(dial_mon_median, how='left', left_on=['client_id', 'month'], right_on=['client_id', 'event_month'])

In [47]:
target_dial.shape

(21797, 9)

In [48]:
target_dial

,mon,target_1,target_2,target_3,target_4,client_id,month,event_month,embedding_sum
0,2022-11-30,0,0,0,0,d89a0712b64d779d9c383298c0cfe0dc27bdf824b9d253...,11,NaN,NaN
1,2022-05-31,0,0,0,0,b1567a620a7525794cd6b7c97259c95a5f8203412040c6...,05,NaN,NaN
2,2022-07-31,0,0,0,0,836850217e1cefa4d145fee8cd1da1ef59a92c98c6fc7b...,07,NaN,NaN
3,2022-03-31,0,0,0,0,7eb598f45524fd5e719c848217a04edc959adbc90c6561...,03,NaN,NaN
4,2022-04-30,0,0,0,0,4ff6cf570f99a6080701874c0d432b046a39bdde515bfc...,04,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21792,2022-06-30,0,0,0,0,7ee6dcd2d61ecf77c6b79caddab160f84dc1ce3b9c2d00...,06,NaN,NaN
21793,2022-07-31,0,0,0,0,7ee6dcd2d61ecf77c6b79caddab160f84dc1ce3b9c2d00...,07,NaN,NaN
21794,2022-12-31,0,0,0,0,2da96394234093af27ba3eb617c27a896c4343706d0dd3...,12,NaN,NaN
21795,2022-06-30,0,0,0,0,fb96b7dd25138a4ee7f0377a862da4827c03a014a5ffca...,06,NaN,NaN


In [49]:
target_dial = target_dial.dropna()
target_dial

,mon,target_1,target_2,target_3,target_4,client_id,month,event_month,embedding_sum
10,2022-07-31,0,0,0,0,93ecad1b4da3a4e973e1f51c57dd60c040314cb1e14a29...,07,07,15.482516
15,2022-09-30,0,0,1,1,b5721051af03c62680bf7c71d9c729c7c57ba0d8eee3b2...,09,09,7.354199
17,2022-03-31,0,0,0,1,1dc6e7e9cff865660a49279660e90c8ad738822762d5dd...,03,03,9.266455
19,2022-04-30,0,0,0,0,a42b3e3660a8c8b26b82924fd1b9808c59691e64339b13...,04,04,16.050846
29,2022-09-30,0,0,1,0,eff5efcde3f28c920e62fa1c67676f15f25d048bc426cf...,09,09,16.712837
...,...,...,...,...,...,...,...,...,...
21760,2022-08-31,0,0,0,0,707b05295b1e9f8b715c1b3b3498be777f96559d02aa14...,08,08,13.199739
21764,2022-03-31,1,0,0,0,b61cf2a1b864b50702f821090fa067f5e7472869b75b4c...,03,03,5.484146
21773,2022-07-31,0,0,0,0,90d74dd0ac4851613bb1de364f2842fcd21bddcf3f6601...,07,07,15.922478
21776,2022-10-31,0,0,0,0,7f27ae36e5e5b0218249708479510eb1451dacb9988d2f...,10,10,13.841089


In [50]:
target_dial.describe()

,mon,target_1,target_2,target_3,target_4,embedding_sum
count,4760,4760.000000,4760.000000,4760.000000,4760.000000,4760.000000
mean,2022-08-11 11:50:55.462184960,0.087395,0.003361,0.044958,0.027521,14.422554
min,2022-02-28 00:00:00,0.000000,0.000000,0.000000,0.000000,-5.717973
25%,2022-06-30 00:00:00,0.000000,0.000000,0.000000,0.000000,11.885395
50%,2022-08-31 00:00:00,0.000000,0.000000,0.000000,0.000000,15.239254
75%,2022-09-30 00:00:00,0.000000,0.000000,0.000000,0.000000,17.368042
max,2022-12-31 00:00:00,1.000000,1.000000,1.000000,1.000000,26.893627
std,NaN,0.282443,0.057886,0.207234,0.163613,4.474153


In [51]:
target_dial = target_dial.drop(['mon', 'event_month'], axis=1)
target_dial.head(1)

,target_1,target_2,target_3,target_4,client_id,month,embedding_sum
10,0,0,0,0,93ecad1b4da3a4e973e1f51c57dd60c040314cb1e14a29...,07,15.482516


Замечетельно! Данные по диалогам пребразовали и проджойнили к таргету. Теперь надо приджойнить данные транзакций

In [52]:
trx

,event_time,amount,client_id,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
0,2022-10-24 16:14:58.662262,98516.000000,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,9,33,11.0,149.0,451.0,869.0,17340.0,31591.0,85.0,482.0,17.0
1,2022-05-30 17:19:09.471755,10989.683594,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,48,43,11.0,149.0,451.0,869.0,17340.0,31591.0,85.0,482.0,17.0
2,2022-05-24 19:07:18.893439,22846.230469,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,37,18,11.0,149.0,451.0,852.0,14606.0,31591.0,85.0,482.0,17.0
3,2022-10-18 14:50:24.836074,16261.271484,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,48,43,11.0,149.0,451.0,869.0,17340.0,31591.0,85.0,482.0,17.0
4,2022-05-31 21:00:09.499419,3799.055908,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,37,18,11.0,149.0,451.0,852.0,14606.0,31591.0,85.0,482.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830534,2022-05-19 11:16:18.755159,273.377411,ef6f662fb508c410948273e5445b326ae5af98f9af4fc0...,25,47,11.0,19.0,344.0,1166.0,30836.0,14090.0,20.0,148.0,67.0
830535,2022-07-22 11:48:43.412586,3502.617676,ef6f662fb508c410948273e5445b326ae5af98f9af4fc0...,25,47,11.0,19.0,344.0,1166.0,30836.0,14090.0,20.0,148.0,67.0
830536,2022-08-03 08:25:45.773491,2380.997314,ef6f662fb508c410948273e5445b326ae5af98f9af4fc0...,25,47,11.0,19.0,344.0,1166.0,30836.0,14090.0,20.0,148.0,67.0
830537,2022-08-16 06:32:15.248168,6283.030273,ef6f662fb508c410948273e5445b326ae5af98f9af4fc0...,25,47,11.0,19.0,344.0,1166.0,30836.0,14090.0,20.0,148.0,67.0


In [53]:
trx['event_time'] = pd.to_datetime(trx['event_time'])  # выделим данные о месяце, когда произошла транзакция 
trx['event_month'] = trx['event_time'].dt.month.astype(str).str.zfill(2)
trx_mon = trx.copy()
trx_mon.head(3)

,event_time,amount,client_id,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32,event_month
0,2022-10-24 16:14:58.662262,98516.000000,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,9,33,11.0,149.0,451.0,869.0,17340.0,31591.0,85.0,482.0,17.0,10
1,2022-05-30 17:19:09.471755,10989.683594,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,48,43,11.0,149.0,451.0,869.0,17340.0,31591.0,85.0,482.0,17.0,05
2,2022-05-24 19:07:18.893439,22846.230469,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,37,18,11.0,149.0,451.0,852.0,14606.0,31591.0,85.0,482.0,17.0,05


In [54]:
trx_mon = trx_mon.drop(['event_time'], axis=1)
trx_mon.head(1)

,amount,client_id,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32,event_month
0,98516.0,230d7d72ff257e1907c1f1a69d03e58621d962ad228798...,9,33,11.0,149.0,451.0,869.0,17340.0,31591.0,85.0,482.0,17.0,10


In [55]:
trx_mon.columns

Index(['amount', 'client_id', 'event_type', 'event_subtype', 'currency',
       'src_type11', 'src_type12', 'dst_type11', 'dst_type12', 'src_type21',
       'src_type22', 'src_type31', 'src_type32', 'event_month'],
      dtype='object')

In [56]:
def mode(series):
    return series.mode()[0] if not series.mode().empty else None

In [57]:
grouped = trx_mon.groupby(['client_id', 'event_month'])

In [58]:
trx_mon_typical = grouped.agg({
    'amount': 'median',
    'event_type': mode,
    'event_subtype': mode,
    'currency': mode,
    'src_type11': mode,
    'src_type12': mode,
    'dst_type11': mode,
    'dst_type12': mode,
    'src_type21': mode,
    'src_type22': mode,
    'src_type31': mode,
    'src_type32': mode
}).reset_index()

In [59]:
trx_mon_typical.shape

(28600, 14)

In [60]:
trx_mon_typical

,client_id,event_month,amount,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
0,009a18f84843dd3175c619e871d9cb8463b2f80abd9b25...,01,12749.378906,37,18,11.0,19.0,902.0,433.0,10049.0,3191.0,70.0,39.0,25.0
1,009a18f84843dd3175c619e871d9cb8463b2f80abd9b25...,02,5568.442383,54,55,11.0,19.0,902.0,433.0,10049.0,3191.0,70.0,39.0,25.0
2,009a18f84843dd3175c619e871d9cb8463b2f80abd9b25...,03,32723.281250,54,55,11.0,19.0,902.0,869.0,10049.0,3191.0,70.0,39.0,25.0
3,009a18f84843dd3175c619e871d9cb8463b2f80abd9b25...,04,23839.916260,37,18,11.0,19.0,902.0,433.0,10049.0,3191.0,70.0,39.0,25.0
4,009a18f84843dd3175c619e871d9cb8463b2f80abd9b25...,05,8522.488281,37,18,11.0,19.0,902.0,433.0,10049.0,3191.0,70.0,39.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28595,ffebd2bf57d9449a02e65b1ce5993681d07156b5edeb8b...,08,14882.339355,54,55,11.0,19.0,902.0,364.0,22652.0,25721.0,48.0,2486.0,26.0
28596,ffebd2bf57d9449a02e65b1ce5993681d07156b5edeb8b...,09,8492.887695,54,55,11.0,19.0,902.0,1166.0,30836.0,25721.0,48.0,2486.0,26.0
28597,ffebd2bf57d9449a02e65b1ce5993681d07156b5edeb8b...,10,4425.326172,54,55,11.0,19.0,902.0,364.0,22652.0,25721.0,48.0,2486.0,26.0
28598,ffebd2bf57d9449a02e65b1ce5993681d07156b5edeb8b...,11,3229.721191,54,55,11.0,19.0,902.0,364.0,22652.0,25721.0,48.0,2486.0,26.0


In [61]:
trx_mon_typical.client_id.nunique()

3072

In [62]:
# Теперь приджойним
target_dial_trx = target_dial.merge(trx_mon_typical, how='left', left_on=['client_id', 'month'], right_on=['client_id', 'event_month'])
target_dial_trx = target_dial_trx.dropna()
target_dial_trx.shape

(3860, 20)

In [63]:
target_dial_trx.head()

,target_1,target_2,target_3,target_4,client_id,month,embedding_sum,event_month,amount,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
1,0,0,1,1,b5721051af03c62680bf7c71d9c729c7c57ba0d8eee3b2...,09,7.354199,09,8953.072754,54.0,55.0,11.0,19.0,344.0,433.0,10049.0,43634.0,28.0,1810.0,51.0
2,0,0,0,1,1dc6e7e9cff865660a49279660e90c8ad738822762d5dd...,03,9.266455,03,1324.851074,37.0,18.0,11.0,19.0,344.0,433.0,10049.0,44828.0,60.0,189.0,4.0
3,0,0,0,0,a42b3e3660a8c8b26b82924fd1b9808c59691e64339b13...,04,16.050846,04,3507.233887,25.0,47.0,11.0,19.0,344.0,1166.0,30836.0,19200.0,63.0,2202.0,76.0
4,0,0,1,0,eff5efcde3f28c920e62fa1c67676f15f25d048bc426cf...,09,16.712837,09,5543.849609,37.0,18.0,11.0,19.0,344.0,364.0,22652.0,3191.0,70.0,2388.0,74.0
5,0,0,0,0,c987955a137230be9dea9b4f57bf5ed37ec9d6943fa55e...,09,17.876408,09,7202.513184,5.0,1.0,11.0,1.0,189.0,433.0,8693.0,4820.0,11.0,844.0,4.0


In [64]:
target_dial_trx = target_dial_trx.drop(['client_id', 'event_month'], axis=1)
target_dial_trx.head(2)

,target_1,target_2,target_3,target_4,month,embedding_sum,amount,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32
1,0,0,1,1,09,7.354199,8953.072754,54.0,55.0,11.0,19.0,344.0,433.0,10049.0,43634.0,28.0,1810.0,51.0
2,0,0,0,1,03,9.266455,1324.851074,37.0,18.0,11.0,19.0,344.0,433.0,10049.0,44828.0,60.0,189.0,4.0


In [65]:
target_dial_trx_fin = target_dial_trx.copy()

In [66]:
target_dial_trx_fin['target_tensor'] = target_dial_trx_fin.apply(lambda row: [row['target_1'], row['target_2'], row['target_3'], row['target_4']], axis=1)

In [67]:
target_dial_trx_fin.head(2)

,target_1,target_2,target_3,target_4,month,embedding_sum,amount,event_type,event_subtype,currency,src_type11,src_type12,dst_type11,dst_type12,src_type21,src_type22,src_type31,src_type32,target_tensor
1,0,0,1,1,09,7.354199,8953.072754,54.0,55.0,11.0,19.0,344.0,433.0,10049.0,43634.0,28.0,1810.0,51.0,"[0, 0, 1, 1]"
2,0,0,0,1,03,9.266455,1324.851074,37.0,18.0,11.0,19.0,344.0,433.0,10049.0,44828.0,60.0,189.0,4.0,"[0, 0, 0, 1]"


In [68]:
target_dial_trx_fin.to_parquet("D:/Файлы/Хакатон_июнь/selected4000_target_dial_trx_fin.parquet", index=False)

In [69]:
# target_dial_trx_fin = target_dial_trx_fin.drop(['target_1', 'target_2', 'target_3', 'target_4'], axis=1)

In [70]:
# target_dial_trx_fin.head(2)  # Полагаю, что месяц может как-то влиять на принятия решения о приобретении продуктов.
# Если так, то банк сможет разработать другие продукты, которые чаще предпочитают брать в другие месяцы

In [71]:
# target_dial_trx_fin.shape  # Умеренно большой датасет

In [72]:
# target_dial_trx_fin.to_parquet("D:/Файлы/Хакатон_июнь/selected4000_target_dial_trx_fin.parquet", index=False)